### Imports and globals

In [52]:
import neo4j

import math
import numpy as np
import pandas as pd

import psycopg2
from geographiclib.geodesic import Geodesic

from IPython.display import display

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [5]:
cursor = connection.cursor()

### Define functions, graphing

In [6]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [7]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [8]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [22]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [9]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [10]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [28]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")

## Define population functions

In [42]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [55]:
def my_station_get_pop(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    #print("\n-------------------------------------------------------------------------------")
    #print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    #print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        #print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    #print("\n-------------------------------------------------------------------------------")
    #print("  Total Population: ", f'{total_population:10,}')
    #print("-------------------------------------------------------------------------------")
    return total_population

### queries

In [13]:
my_neo4j_wipe_out_database()

In [19]:
#stations
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)

In [23]:
#1 way lines
connection.rollback()

query = """

select station, line from lines;

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    my_neo4j_create_node(f'{line} {station}')
    my_neo4j_create_relationship_one_way(f'depart {station}', f'{line} {station}', 0)
    my_neo4j_create_relationship_one_way(f'{line} {station}', f'arrive {station}', 0)

In [24]:
#transfers between lines
connection.rollback()

query = """

select 
  l.*,
  s.transfer_time
from (
  select 
    a.station,
    a.line as from_line,
    b.line as to_line
  from lines a, lines b
  where a.station=b.station and a.line!=b.line 
) l
join stations s
on l.station=s.station
;

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    my_neo4j_create_relationship_one_way(f'{from_line} {station}', f'{to_line} {station}', transfer_time)

In [25]:
#create two way connected graph for trains

connection.rollback()

query = """

with t1 as (
  select 
    a.line,
    b.station as from_station,
    a.station as to_station
  from lines a, lines b
  where a.line=b.line and a.sequence=b.sequence+1 
)

select 
  a.*,
  b.travel_time as travel_time_in_seconds
from t1 a
join travel_times b
on 
  a.from_station=b.station_1 and a.to_station=b.station_2
  or a.from_station=b.station_2 and a.to_station=b.station_1
order by 1, 4
;

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time_sec = int(row[3])
    
    my_neo4j_create_relationship_two_way(f'{line} {from_station}', f'{line} {to_station}', travel_time_sec)

In [26]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 215
  Relationships: 656
-------------------------


## Centrality

In [29]:
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,arrive 12th Street,[Station]
1,arrive 16th Street Mission,[Station]
2,arrive 19th Street,[Station]
3,arrive 24th Street Mission,[Station]
4,arrive Antioch,[Station]
...,...,...
210,yellow SFO,[Station]
211,yellow San Bruno,[Station]
212,yellow South San Francisco,[Station]
213,yellow Walnut Creek,[Station]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,blue 16th Street Mission,[Station],LINK,arrive 16th Street Mission,[Station]
1,blue 16th Street Mission,[Station],LINK,blue 24th Street Mission,[Station]
2,blue 16th Street Mission,[Station],LINK,blue Civic Center,[Station]
3,blue 16th Street Mission,[Station],LINK,green 16th Street Mission,[Station]
4,blue 16th Street Mission,[Station],LINK,red 16th Street Mission,[Station]
...,...,...,...,...,...
651,yellow West Oakland,[Station],LINK,blue West Oakland,[Station]
652,yellow West Oakland,[Station],LINK,green West Oakland,[Station]
653,yellow West Oakland,[Station],LINK,red West Oakland,[Station]
654,yellow West Oakland,[Station],LINK,yellow 12th Street,[Station]


-------------------------
  Density: 0.0
-------------------------


In [32]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK')"
session.run(query)

In [34]:
query = """

CALL gds.degree.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as degree
ORDER BY degree DESC, name

"""

df = my_neo4j_run_query_pandas(query)

In [76]:
#list of stations with max centrality
top_stations = df[df['degree'] == df['degree'].max(axis = 0)]
top_stations

,name,degree
0,blue 16th Street Mission,6.0
1,blue 24th Street Mission,6.0
2,blue Balboa Park,6.0
3,blue Civic Center,6.0
4,blue Coliseum,6.0
5,blue Embarcadero,6.0
6,blue Glen Park,6.0
7,blue Montgomery Street,6.0
8,blue Powell Street,6.0
9,blue West Oakland,6.0


In [77]:
top_stations['lookup_name'] = top_stations['name'].str.split(n=1).str[1]

<ipython-input-77-3773c7996b83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_stations['lookup_name'] = top_stations['name'].str.split(n=1).str[1]


In [78]:
del top_stations['name']
top_stations = top_stations.drop_duplicates()

In [79]:
#add population within X distance of each top station
top_stations['population'] = top_stations['lookup_name'].apply(lambda x: my_station_get_pop(x, 2))

<ipython-input-79-024a87db774d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_stations['population'] = top_stations['lookup_name'].apply(lambda x: my_station_get_pop(x, 2))


In [83]:
#sort by top population
top_stations.sort_values(by=['population'],ascending=False)

,degree,lookup_name,population
3,6.0,Civic Center,347795
0,6.0,16th Street Mission,339093
1,6.0,24th Street Mission,315201
8,6.0,Powell Street,303556
6,6.0,Glen Park,287877
7,6.0,Montgomery Street,268802
2,6.0,Balboa Park,253123
25,6.0,Daly City,231056
5,6.0,Embarcadero,210921
4,6.0,Coliseum,123040
